In [1]:
from __future__ import print_function
import pandas as pd
import gspread_pandas
from oauth2client.service_account import ServiceAccountCredentials
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen
import json
from tqdm import tqdm
from time import sleep
import yaml

In [2]:
config=gspread_pandas.conf.get_config(conf_dir='/home/dev-cory/learning_english/credential/',file_name='client_secret_english_crawl.json')
#gspread_pandas.conf.get_config
gspread_pandas.conf.get_config
with open('./credential/learning_english.yaml','r') as ymlfile:
    cfg = yaml.load(ymlfile,Loader=yaml.BaseLoader)
app_id = cfg['OxfordAPI']['app_id']
app_key = cfg['OxfordAPI']['app_key']
language = 'en'

In [3]:
spread = gspread_pandas.Spread('Oxford vocabulary',config=config)

In [4]:
df=spread.sheet_to_df(sheet='vocab')

In [5]:
df

,Word,Importance,Meaning in Japanese,Meaning in English,Example,Origin,phonic,mp3
ru,,,,,,,,
1,retaliate,☆,仕返しする,1: make an attack in return for a similar atta...,It is still unclear who started the violence b...,early 17th century: from Latin retaliat-‘retur...,rɪˈtalɪeɪt,http://audio.oxforddictionaries.com/en/mp3/ret...
2,detain,3,拘束する,keep (someone) from proceeding by holding them...,"Image caption Yang Hengjun, a popular blogger ...",late Middle English (in the sense ‘be afflicte...,dɪˈteɪn,http://audio.oxforddictionaries.com/en/mp3/det...
3,resilient,3,はね返る、弾力のある、たちまち元気を回復する、快活な、溌刺(はつらつ)とした,(of a person or animal) able to withstand or r...,,mid 17th century: from Latin resilient-‘leapin...,rɪˈzɪlɪənt,http://audio.oxforddictionaries.com/en/mp3/res...
4,lineage,4,血統、系統、家柄,direct descent from an ancestor; ancestry or p...,,"Middle English: from Old French lignage, from ...",ˈlɪnɪɪdʒ,http://audio.oxforddictionaries.com/en/mp3/lin...
5,migration,1,移住、転住、渡り、移住者群、移動する動物の群,1: seasonal movement of animals from one regio...,,,mʌɪˈɡreɪʃ(ə)n,http://audio.oxforddictionaries.com/en/mp3/mig...
...,...,...,...,...,...,...,...,...
1835,dignitary,,,,Visiting dignitaries brush shoulders with wome...,,,
1836,municipality,,,,Another problem is the split between native ho...,,,
1837,evict,,,,Each year from November 1st to March 31st Fran...,,,


In [6]:
 df.loc['4','Meaning in Japanese']

'血統、系統、家柄'

In [7]:
def weblio_crawler(word_id):
    url ='https://ejje.weblio.jp/content/'+word_id
    try:
        html_page = urlopen(url)
        soup = BeautifulSoup(html_page)
        text=soup.find('td',attrs={"class", "content-explanation ej"})
        answer=text.contents[0]
    except:
        answer=''
    return answer

In [8]:
def Oxford_definition_caller(word_id,app_id=app_id,app_key=app_key,language ='en'):
    url = 'https://od-api.oxforddictionaries.com:443/api/v2/entries/'  + language + '/'  + word_id.lower()
    urlFR = 'https://od-api.oxforddictionaries.com:443/api/v2/stats/frequency/word/'  + language + '/?corpus=nmc&lemma=' + word_id.lower()
    try:
        r = requests.get(url, headers = {'app_id' : app_id, 'app_key' : app_key})
        info=r.json()
        x=1
        answer=''
        for i in info["results"][0]['lexicalEntries'][0]["entries"][0]["senses"]:
            answer=answer +str(x)+': '+str(i["definitions"][0])+'  '
            x+=1
            sleep(5)
    except:
        answer=''
    return answer

In [9]:
def Oxford_caller(word_id,app_id=app_id,app_key=app_key,language ='en'):
    url = 'https://od-api.oxforddictionaries.com:443/api/v2/entries/'  + language + '/'  + word_id.lower()
    urlFR = 'https://od-api.oxforddictionaries.com:443/api/v2/stats/frequency/word/'  + language + '/?corpus=nmc&lemma=' + word_id.lower()
    try:
        r = requests.get(url, headers = {'app_id' : app_id, 'app_key' : app_key})
        info=r.json()
        sleep(5)
    except Exception as e:
        print(e)
        info=''
    return info

In [10]:
def Oxford_example_caller(word_id,app_id=app_id,app_key=app_key,language ='en'):
    url = 'https://od-api.oxforddictionaries.com:443/api/v2/entries/'  + language + '/'  + word_id.lower()
    urlFR = 'https://od-api.oxforddictionaries.com:443/api/v2/stats/frequency/word/'  + language + '/?corpus=nmc&lemma=' + word_id.lower()
    try:
        r = requests.get(url, headers = {'app_id' : app_id, 'app_key' : app_key})
        info=r.json()
        x=1
        answer=''
        for i in info["results"][0]['lexicalEntries'][0]["entries"][0]["senses"]:
            answer=answer +str(x)+': '+str(i["examples"][0]["text"])+'  '
            x+=1
            sleep(5)
    except:
        answer=''
    return answer

In [11]:
def Oxford_info_processor(info,target):
    try:
        answer=''
        if target=='meaning':
            x=1
            for i in info["results"][0]['lexicalEntries'][0]["entries"][0]["senses"]:
                answer=answer +str(x)+': '+str(i["definitions"][0])+' \r\n'
                x+=1       
        elif target=='phonic':
            answer=info["results"][0]['lexicalEntries'][0]["pronunciations"][0]["phoneticSpelling"]
        elif target=='example':
            x=1
            for i in info["results"][0]['lexicalEntries'][0]["entries"][0]["senses"]:
                answer=answer +str(x)+': '+str(i["examples"][0]["text"])+' \r\n'
                x+=1
        elif target=='mp3':
            answer=info["results"][0]['lexicalEntries'][0]["pronunciations"][0]["audioFile"]
        elif target=='origin':
            answer=info["results"][0]['lexicalEntries'][0]['entries'][0]['etymologies'][0]
    except Exception as e:
        print(e)
        answer=''
    return answer

In [12]:
for index in tqdm(df.index):
    word_id=df.loc[index,'Word']
    word_id=word_id.strip(' ')
    word_id=word_id.lower()
    if df.loc[index,'Meaning in Japanese']=='':
        df.loc[index,'Meaning in Japanese']=weblio_crawler(word_id)
    if df.loc[index,'Meaning in English']=='':
        info=Oxford_caller(word_id)
        df.loc[index,'Meaning in English']=Oxford_info_processor(info,'meaning')
        #print(Oxford_info_processor(info,'meaning'))
        if df.loc[index,'Example']=='':
            df.loc[index,'Example']=Oxford_info_processor(info,'example')
        if df.loc[index,'Origin']=='':
            df.loc[index,'Origin']=Oxford_info_processor(info,'origin')
            df.loc[index,'phonic']=Oxford_info_processor(info,'phonic')
            #print(Oxford_info_processor(info,'phonic'))
            df.loc[index,'mp3']=Oxford_info_processor(info,'mp3')
    '''
    if df.loc[index,'Example']=='':
        info=Oxford_caller(word_id)
        df.loc[index,'Example']=Oxford_info_processor(info,'example')
        if df.loc[index,'Origin']=='':
            df.loc[index,'Origin']=Oxford_info_processor(info,'origin')
            df.loc[index,'phonic']=Oxford_info_processor(info,'phonic')
            df.loc[index,'mp3']=Oxford_info_processor(info,'mp3')
    if df.loc[index,'Origin']=='':
        info=Oxford_caller(word_id)
        df.loc[index,'Origin']=Oxford_info_processor(info,'origin')
        df.loc[index,'phonic']=Oxford_info_processor(info,'phonic')
        df.loc[index,'mp3']=Oxford_info_processor(info,'mp3')
    '''
    
        

  2%|▏         | 32/1822 [00:05<05:23,  5.53it/s]

'definitions'
'examples'


  2%|▏         | 41/1822 [00:11<09:22,  3.17it/s]

'definitions'
'examples'
'etymologies'


  7%|▋         | 134/1822 [00:17<06:43,  4.18it/s]

...

'results'
'results'
'results'
'results'


100%|█████████▉| 1817/1822 [32:22<00:38,  7.65s/it]

'etymologies'


100%|██████████| 1822/1822 [33:00<00:00,  1.09s/it]

'etymologies'


In [13]:
spread.df_to_sheet(df, index=True, sheet='result', start='A1', replace=True)

In [ ]:
df